# 3D Medical Image Segmentation with nnUNetV2 (Lower Limb)

This notebook demonstrates a complete workflow for performing 3D medical image segmentation, specifically targeting lower limb structures, using the powerful `nnUNetV2` framework. The process involves:

1.  **Environment Setup**: Installing necessary libraries.
2.  **Data Preparation**: Mounting Google Drive, defining paths, and organizing input data and model checkpoints.
3.  **Model Loading**: Initializing and loading a pre-trained `nnUNetV2` model.
4.  **Inference**: Executing the segmentation prediction on a raw medical image.
5.  **Result Analysis**: Calculating and displaying volumetric statistics of the segmented anatomical structures and saving the results.

This workflow is designed to be easily adaptable for various medical image segmentation tasks using `nnUNetV2` models.

### Environment Setup

This section handles the installation of the required Python libraries. We will install `nnunetv2`, the core framework for 3D medical image segmentation, and `nibabel`, which is essential for reading and writing neuroimaging file formats such as NIfTI (.nii.gz).

In [24]:
import subprocess
import sys
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "nnunetv2", "nibabel"], check=True)
print("✅ Installation complete\n")

✅ Installation complete



## Data Preparation: Mounting Google Drive

This section handles the mounting of Google Drive, which is crucial for this notebook's operation. By mounting Google Drive, we gain access to essential input files, including the raw NIfTI medical image for segmentation, the pre-trained `nnUNetV2` model checkpoint, and configuration files (`dataset.json`, `plans.json`). It also allows us to save the final segmentation results directly back to Google Drive for persistent storage and future use.

In [25]:
from google.colab import drive
print("📁 Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ Drive mounted\n")

📁 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted



### Data Preparation: Configuration

This section sets up the environment and prepares all necessary files for the `nnUNetV2` inference. It defines critical paths for input data, model checkpoints, and output results, both on Google Drive and within the Colab working directory. It also creates the required directory structure and copies the model and input files from Google Drive to the local working environment, ensuring that the `nnUNetV2` predictor has access to all its dependencies.

In [26]:
import os
import shutil
from pathlib import Path
import torch

print("=" * 70)
print("⚙️  CONFIGURATION")
print("=" * 70)

# Files on Google Drive
INPUT_FILE = "/content/drive/MyDrive/SMIR.Lower_limb.051Y.F.CT.168-Pelvis-Thighs.nii.gz"
CHECKPOINT_FILE = "/content/drive/MyDrive/checkpoint_best.pth"
DATASET_JSON = "/content/drive/MyDrive/dataset.json"
PLANS_JSON = "/content/drive/MyDrive/plans.json"

# Working directories
WORK_DIR = "/content/nnunet_inference"
MODEL_DIR = f"{WORK_DIR}/model"
INPUT_DIR = f"{WORK_DIR}/input"
OUTPUT_DIR = f"{WORK_DIR}/output"

# Create structure (include fold_0 for the checkpoint)
FOLD_DIR = f"{MODEL_DIR}/fold_0"
for d in [MODEL_DIR, FOLD_DIR, INPUT_DIR, OUTPUT_DIR]:
    Path(d).mkdir(parents=True, exist_ok=True)

# Verify source files
print("\n🔍 Verifying files:")
files_ok = True
for name, path in [
    ("Input NIfTI", INPUT_FILE),
    ("Checkpoint", CHECKPOINT_FILE),
    ("Dataset JSON", DATASET_JSON),
    ("Plans JSON", PLANS_JSON)
]:
    if os.path.exists(path):
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"  ✅ {name}: {size_mb:.1f} MB")
    else:
        print(f"  ❌ {name}: NOT FOUND - {path}")
        files_ok = False

if not files_ok:
    raise FileNotFoundError("Missing files on Google Drive!")

# Copy files to the working folder
print("\n📋 Preparing files...")
shutil.copy(PLANS_JSON, f"{MODEL_DIR}/plans.json")
shutil.copy(CHECKPOINT_FILE, f"{FOLD_DIR}/checkpoint_best.pth")  # In fold_0/
shutil.copy(INPUT_FILE, f"{INPUT_DIR}/case_001_0000.nii.gz")
if os.path.exists(DATASET_JSON):
    shutil.copy(DATASET_JSON, f"{MODEL_DIR}/dataset.json")

print("✅ Files prepared\n")

⚙️  CONFIGURATION

🔍 Verifying files:
  ✅ Input NIfTI: 72.0 MB
  ✅ Checkpoint: 341.6 MB
  ✅ Dataset JSON: 0.0 MB
  ✅ Plans JSON: 0.0 MB

📋 Preparing files...
✅ Files prepared



### Model Loading

This section focuses on initializing and loading the pre-trained `nnUNetV2` model. It detects the available computing device (GPU if available, otherwise CPU) and then sets up the `nnUNetPredictor` with specific inference parameters. Finally, it loads the model weights from the provided checkpoint file into the predictor, preparing it for the segmentation task.

In [27]:
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor

print("=" * 70)
print("🚀 MODEL LOADING")
print("=" * 70)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# Initialize the predictor
print("\n📦 Initializing predictor...")
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    perform_everything_on_device=True,
    device=device,
    verbose=False,
    allow_tqdm=True
)

# Load the model from the checkpoint
print("🔄 Loading checkpoint...")
predictor.initialize_from_trained_model_folder(
    MODEL_DIR,
    use_folds=(0,),
    checkpoint_name='checkpoint_best.pth'
)

print("✅ Model loaded successfully!\n")

🚀 MODEL LOADING
🖥️  Device: cuda
   GPU: Tesla T4

📦 Initializing predictor...
🔄 Loading checkpoint...
✅ Model loaded successfully!



### Inference

This section performs the actual segmentation of the medical image using the loaded `nnUNetV2` model. The `predict_from_files` method of the `nnUNetPredictor` is invoked, taking the prepared input image from the specified input directory and generating the segmentation mask, which is then saved to the output directory. This step leverages the power of the pre-trained model to automatically identify and delineate anatomical structures within the 3D medical scan.

In [28]:
import nibabel as nib
import numpy as np

print("=" * 70)
print("🧠 INFERENCE")
print("=" * 70)

print(f"📂 Input: {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")

# Launch inference
print("\n🚀 Launching inference...")
predictor.predict_from_files(
    list_of_lists_or_source_folder=INPUT_DIR,
    output_folder_or_list_of_truncated_output_files=OUTPUT_DIR,
    save_probabilities=False,
    overwrite=True,
    num_processes_preprocessing=2,
    num_processes_segmentation_export=2,
)

print("✅ Inference complete!\n")

🧠 INFERENCE
📂 Input: /content/nnunet_inference/input
📂 Output: /content/nnunet_inference/output

🚀 Launching inference...
There are 1 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting case_001:
perform_everything_on_device: True


100%|██████████| 252/252 [07:28<00:00,  1.78s/it]


sending off prediction to background worker for resampling and export
done with case_001
✅ Inference complete!



### Statistics and Result Saving

This final section is dedicated to analyzing the segmentation results and saving them. It loads the generated NIfTI segmentation file, calculates volumetric statistics for each detected anatomical structure (bones) based on predefined labels, and then saves the final segmentation output to Google Drive for future reference and persistent storage.

In [29]:
import nibabel as nib
import numpy as np

print("=" * 70)
print("📊 STATISTICS")
print("=" * 70)

# Labels
LABELS = {
    0: "background",
    1: "Femur_L",
    2: "Femur_R",
    3: "Hip_L",
    4: "Hip_R",
    5: "Patella_L",
    6: "Patella_R",
    7: "Sacrum",
    8: "Threshold-200-MAX"
}

# Find the output file
output_files = list(Path(OUTPUT_DIR).glob("*.nii.gz"))
if output_files:
    output_file = output_files[0]
    print(f"✅ Segmentation: {output_file.name}\n")

    # Load and analyze
    nii = nib.load(str(output_file))
    seg_data = nii.get_fdata()
    spacing = nii.header.get_zooms()

    print(f"📏 Dimensions: {seg_data.shape}")
    print(f"📏 Spacing: {spacing}")
    print(f"\n🦴 Detected Bones:")

    # Statistics per bone
    for label_id in np.unique(seg_data):
        if label_id > 0:
            voxel_count = int(np.sum(seg_data == label_id))
            volume_mm3 = float(voxel_count * np.prod(spacing))
            volume_cm3 = volume_mm3 / 1000
            label_name = LABELS.get(int(label_id), f"Unknown_{int(label_id)}")
            print(f"   • {label_name:20s}: {voxel_count:8,} voxels ({volume_cm3:8.2f} cm³)")

    # Save to Google Drive
    output_on_drive = "/content/drive/MyDrive/segmentation_result.nii.gz"
    shutil.copy(str(output_file), output_on_drive)

    print(f"\n💾 Result saved on Google Drive:")
    print(f"   {output_on_drive}")

else:
    print("❌ No output file found!")

print("\n" + "=" * 70)
print("✅ COMPLETED!")
print("=" * 70)

📊 STATISTICS
✅ Segmentation: case_001.nii.gz

📏 Dimensions: (312, 205, 889)
📏 Spacing: (np.float32(0.9765625), np.float32(0.9765625), np.float32(0.6997555))

🦴 Detected Bones:
   • Femur_L             :  943,041 voxels (  629.33 cm³)
   • Hip_L               :  730,025 voxels (  487.17 cm³)
   • Patella_L           :   51,400 voxels (   34.30 cm³)
   • Patella_R           :      930 voxels (    0.62 cm³)
   • Sacrum              :  195,121 voxels (  130.21 cm³)
   • Threshold-200-MAX   :  236,533 voxels (  157.85 cm³)

💾 Result saved on Google Drive:
   /content/drive/MyDrive/segmentation_result.nii.gz

✅ COMPLETED!
